In [1]:
import torch
from torch import nn
import numpy
import sentencepiece

In [2]:
the_device = torch.device("cuda")
print("device: ", the_device)

device:  cuda


In [3]:
#temp = numpy.load("train_.npy")
#the_array = torch.tensor(temp, dtype = torch.int32)
#torch.save(the_array, "torch_train.pt")

In [3]:
train_it = torch.load("torch_train.pt")

C:\Users\Alper\AppData\Local\Temp\ipykernel_35216\26094260.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_it = torch.load("torch_train.pt")


In [4]:
sp = sentencepiece.SentencePieceProcessor("omer_s.model")


In [ ]:
import math
import torch
import torch.nn as nn
from torch.optim import AdamW
import sentencepiece as spm

from data import load_ids, batchify, get_batch
from model import LSTMLM

def detach_hidden(h):
    if h is None:
        return None
    return tuple(t.detach() for t in h)

def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("device:", device)

    sp = spm.SentencePieceProcessor(model_file="omer_s.model")
    vocab_size = sp.get_piece_size()
    print("vocab_size:", vocab_size)

    #set hyperparameters
    batch_size = 32
    seq_len = 128
    emb_dim = 256
    hidden_dim = 512
    num_layers = 2
    dropout = 0.25
    lr = 2e-3
    epochs = 500
    clip = 1.0

    raw = load_ids("torch_train.pt")
    data = batchify(raw, batch_size, device)
    
    print("data shape [T,B]:", tuple(data.shape))

    model = LSTMLM(vocab_size, emb_dim, hidden_dim, num_layers, dropout).to(device)
    ckpt = torch.load("ckpt_ep500.pt", map_location=device)
    model = LSTMLM(
        ckpt["vocab_size"],
        ckpt["emb_dim"],
        ckpt["hidden_dim"],
        ckpt["num_layers"],
        ckpt["dropout"],
    ).to(device)
    model.load_state_dict(ckpt["model"])


    criterion = nn.CrossEntropyLoss()
    opt = AdamW(model.parameters(), lr=lr)

    for ep in range(1, epochs + 1):
        model.train()
        h = None
        total_loss = 0.0
        steps = 0

        for i in range(0, data.size(0) - 1, seq_len):
            x, y = get_batch(data, i, seq_len)
            h = detach_hidden(h)

            opt.zero_grad(set_to_none=True)
            logits, h = model(x, h)

            loss = criterion(logits.reshape(-1, vocab_size), y.reshape(-1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()

            total_loss += loss.item()
            steps += 1

        avg = total_loss / max(1, steps)
        ppl = math.exp(avg) if avg < 20 else float("inf")
        if (ep % 10 == 0):
            print(f"epoch {ep:02d} | loss {avg:.4f} | ppl {ppl:.2f}")
        
            torch.save({
                "model": model.state_dict(),
                "vocab_size": vocab_size,
                "emb_dim": emb_dim,
                "hidden_dim": hidden_dim,
                "num_layers": num_layers,
                "dropout": dropout
            }, f"ckpt_ep{ep:02d}.pt")

if __name__ == "__main__":
    main()


device: cuda
vocab_size: 16000
data shape [T,B]: (5711, 32)


c:\Users\Alper\Desktop\formyself\RNNs\data.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(path).to(torch.long)
C:\Users\Alper\AppData\Local\Temp\ipyke

epoch 10 | loss 0.3371 | ppl 1.40
epoch 20 | loss 0.3270 | ppl 1.39
epoch 30 | loss 0.3208 | ppl 1.38
epoch 40 | loss 0.3112 | ppl 1.37
epoch 50 | loss 0.3055 | ppl 1.36
epoch 60 | loss 0.2934 | ppl 1.34
epoch 70 | loss 0.2899 | ppl 1.34
epoch 80 | loss 0.2765 | ppl 1.32
epoch 90 | loss 0.2724 | ppl 1.31
epoch 100 | loss 0.2633 | ppl 1.30
epoch 110 | loss 0.2605 | ppl 1.30
epoch 120 | loss 0.2545 | ppl 1.29
epoch 130 | loss 0.2461 | ppl 1.28
epoch 140 | loss 0.2442 | ppl 1.28
epoch 150 | loss 0.2433 | ppl 1.28
epoch 160 | loss 0.2412 | ppl 1.27
epoch 170 | loss 0.2342 | ppl 1.26
epoch 180 | loss 0.2322 | ppl 1.26
epoch 190 | loss 0.2229 | ppl 1.25
epoch 200 | loss 0.2176 | ppl 1.24
epoch 210 | loss 0.2123 | ppl 1.24
epoch 220 | loss 0.2091 | ppl 1.23
epoch 230 | loss 0.2054 | ppl 1.23
epoch 240 | loss 0.2013 | ppl 1.22
epoch 250 | loss 0.1963 | ppl 1.22
epoch 260 | loss 0.1959 | ppl 1.22
epoch 270 | loss 0.1916 | ppl 1.21
epoch 280 | loss 0.1859 | ppl 1.20
epoch 290 | loss 0.1821 | ppl

In [7]:
#TRAINED FOR 350 EPOCHS -> 0.51 loss, 1.67 perplexity

In [ ]:
import torch
import sentencepiece as spm
from model import LSTMLM

@torch.no_grad()
def generate(model, sp, prompt: str, max_new_tokens=300, temperature=0.9, top_k=50, device="cpu"):
    model.eval()

    ids = sp.encode(prompt, out_type=int)
    x = torch.tensor(ids, dtype=torch.long, device=device).unsqueeze(1)  # [S,1]

    h = None
    _, h = model(x, h) #read the prompt, construct the hidden state
    cur = x[-1:] #

    out_ids = ids[:]
    for _ in range(max_new_tokens):
        logits, h = model(cur, h)
        logits = logits[0, 0] / max(1e-8, temperature)

        if top_k and top_k > 0:
            v, ix = torch.topk(logits, top_k)
            probs = torch.softmax(v, dim=-1)
            next_local = torch.multinomial(probs, 1).item()
            next_id = ix[next_local].item()
        else:
            probs = torch.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, 1).item()

        out_ids.append(next_id)
        cur = torch.tensor([[next_id]], dtype=torch.long, device=device)

    return sp.decode(out_ids)

def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    sp = spm.SentencePieceProcessor(model_file="omer_s.model")

    ckpt = torch.load("LSTM_1_layers.pt", map_location=device)
    model = LSTMLM(
        ckpt["vocab_size"],
        ckpt["emb_dim"],
        ckpt["hidden_dim"],
        ckpt["num_layers"],
        ckpt["dropout"],
    ).to(device)
    model.load_state_dict(ckpt["model"])

    print(generate(
        model, sp,
        prompt="Yarın arifeydi. Öbür günkü bayram için hazırlanan beyaz kurbanlar, küçük Grijgal palangasının etrafında otluyorlardı.",
        max_new_tokens=500,
        temperature=0.9,
        top_k=50,
        device=device
    ))

if __name__ == "__main__":
    main()


C:\Users\Alper\AppData\Local\Temp\ipykernel_35216\2901446395.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("LSTM_1_layers.pt", map_location=device)


Yarın arifeydi. Öbür günkü bayram için hazırlanan beyaz kurbanlar, küçük Grijgal palangasının etrafında otluyorlardı. Mıstık'ın beğenmeyip bıraktığı en miskin, en hasta, en ihtiyar hayvanları bile alıyor, bir gün içinde gençleştiriyor, kuyruğunu, yelesini kesmeden, şeklini değiştiriyor, gözlerini parlatıyor, şahlandırıyordu. Mıstık, henüz geldiği kasabanın hanından girerken yine bu herifi gördü. Yeni bir zarara uğramış gibi birdenbire canı sıkıldı. Ama bozuntuya vermedi: — Merhaba Molla! dedi. — Merhaba?.. Şimdiye kadar hiç konuşmamışlardı. — Hayvan almaya mı geldin? — Sana ne? . . . . . . . — Neye geldimse geldim. Mıstık, kirli zayıf elini seyrek sarı bıyıklarına kaldırdı. Çakır gözleri bakacak yer bulamadı. Renksiz dudaklarını kısarak gülümsedi: — Ortak olalım be, dedi. — Olalım... Molla da gülümsedi. Döndüler. Hanın avlusuna doğru yan yana yürüdüler. Kahvenin önündeki eski peykeye oturdular. Ayaklarının dibinde iri, alacalı bir tavuk "gut, gut, gut" diye civcivlerini gezdiriyordu. P

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
sp = spm.SentencePieceProcessor(model_file="omer_s.model")

ckpt = torch.load("LSTM_1_layers.pt", map_location=device)
model = LSTMLM(
    ckpt["vocab_size"],
    ckpt["emb_dim"],
    ckpt["hidden_dim"],
    ckpt["num_layers"],
    ckpt["dropout"],
).to(device)
model.load_state_dict(ckpt["model"])

print(generate(
    model, sp,
    prompt="Yarın arifeydi. Öbür günkü bayram için hazırlanan beyaz kurbanlar, küçük Grijgal palangasının etrafında otluyorlardı.",
    max_new_tokens=30,
    temperature=0.9,
    top_k=50,
    device=device
))

C:\Users\Alper\AppData\Local\Temp\ipykernel_35216\730891572.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("LSTM_1_layers.pt", map_location=device)


Yarın arifeydi. Öbür günkü bayram için hazırlanan beyaz kurbanlar, küçük Grijgal palangasının etrafında otluyorlardı. Mıstık'ın beğenmeyip bıraktığı en miskin, en hasta, en ihtiyar hayvanları bile alıyor, bir gün içinde gençleştiriyor, kuyruğunu, yelesini kesmeden


### Prompt: "Yarın arifeydi. Öbür günkü bayram için hazırlanan beyaz kurbanlar, küçük Grijgal palangasının etrafında otluyorlardı."

<div style="display:flex; gap:40px">

<div>

### 1-Layer LSTM
Yarın arifeydi. Öbür günkü bayram için hazırlanan beyaz kurbanlar, küçük Grijgal palangasının etrafında otluyorlardı. Mıstık'ın beğenmeyip bıraktığı en miskin, en hasta, en ihtiyar hayvanları bile alıyor, bir gün içinde gençleştiriyor, kuyruğunu, yelesini kesmeden, şeklini değiştiriyor, gözlerini parlatıyor, şahlandırıyordu. Mıstık, henüz geldiği kasabanın hanından girerken yine bu herifi gördü. Yeni bir zarara uğramış gibi birdenbire canı sıkıldı. Ama bozuntuya vermedi: — Merhaba Molla! dedi. — Merhaba?.. Şimdiye kadar hiç konuşmamışlardı. — Hayvan almaya mı geldin? — Sana ne? . . . . . . . — Neye geldimse geldim. Mıstık, kirli zayıf elini seyrek sarı bıyıklarına kaldırdı. Çakır gözleri bakacak yer bulamadı. Renksiz dudaklarını kısarak gülümsedi: — Ortak olalım be, dedi. — Olalım... Molla da gülümsedi. Döndüler. Hanın avlusuna doğru yan yana yürüdüler. Kahvenin önündeki eski peykeye oturdular. Ayaklarının dibinde iri, alacalı bir tavuk "gut, gut, gut" diye civcivlerini gezdiriyordu. Pazar yarındı. Mıstık koynundan tütün kesesini çıkardı. Molla'ya uzatırken, peykenin yanındaki pencereden içeriye bağırdı: — Bize iki kahve getir. Molla: — Ben oruçluyum! dedi. Mıstık anlamadı: — Ramazanda mıyız yahu? — Hayır. — Üç aylarda mıyız? — Hayır. — Ey, bu ne orucu? — Ben bütün yıl bir gün yer, bir gün tutarım! — Sahi mi? — Vallahi... Mıstık tütün kesesini tekrar koynuna soktu. Eğildi, camsız pencereden kahveciye: — İstemez, kahveleri yapma, diye seslendi. İçinden, "Bu gebeşin kafasına ben bir külah geçiririm!" dedi. Kendisinin sofuluğundan, küçükken hafızlığa çalıştığından, ama hastalandığı için vazgeçtiğinden, babasının yirmi yedi defa hacca gittiğinden bahsetti. Molla yere bakarak dinliyor, başını sallıyor, inanıyor, Rumelililerin sağlam Müslüman olduklarını söylüyordu. Mıstık sordu: — Sen nerelisin? — Kayserili. — Kayseri nerede? — Bu tarafta... Molla, kısa parmaklı tombul eliyle hanın kapısını gösteriyordu. Mıstık, geldiği ciheti hatırlayarak: — Konya tarafında mı? diye sordu. — Hayır canım, daha yukarılarda... Mıstık, Kayseri'nin nerede, hem de ne olduğunu pek iyi biliyordu. Rumeli'de bıraktığı çiftlikleri de anlattıktan sonra yaptığı kapıyı kâfi gördü. İşlere geçti. Konuştular, anlaştılar. O günden sonra ortaklığa karar verdiler. Kâra, zarara, sermayeye ortak oluyorlardı. Mıstık yine içinden, "Ben sana bir külah giydireyim de, gör!" dedi. Ertesi gün


</div>

<div>

### 2-Layer LSTM
Yarın arifeydi. Öbür günkü bayram için hazırlanan beyaz kurbanlar, küçük Grijgal palangasının etrafında otluyorlardı. İşleri ansızın giydiriyordu. 🙝🙟 Biraz sonra... Siyah ağaçların arkasından doğan kan renginde bir ay, bizim gözlerini bayraktıyordu. Dar masanın orta yerinde duruyordu. Elini uzattı. — Bu büyük günden bana ne kadar adam olduğumu görmem. İşte ben de her sabah bir şey olacak. — Ne vakit?.. — Mesela... Niyazi misalini söyleyemez. Sokaktan şiddetli, keskin, gür, canlı, latif, parlak, ahenkli bir ses gelir. "Dünya değişti. Eski günler geçti. Merhamet, mürüvvet, insaniyet kalmadı. Herkes keyfinde, eğlencesinde, kimse kimseyi düşünmez oldu. Bu ne haldir?" Birbirlerinin yüzlerine bakışırlar. Gayri ihtiyarî, sârî bir hareketle, tellenmiş cigaralarını önlerindeki tablada söndürürler: — Bu ne? — Bilmem. Sokaktan gelen ahenkli ses aynı şiddet, aynı letafet, aynı azamet, aynı belâgat le devam eder: "...Dünya bir cife dir. Hayf onu isteyen köpeklere! Uyanın, kâinata ibretle bakın. Fâni olan şeylere aldanmayın." Neşet bir eliyle başını, diğer eliyle sarı pijamasının üstünden kalçasını kaşır. Yüzünü ekşitir. Niyazi alt dudağını ısırarak yavaş yavaş doğrulur: — Bu ne be? — Saçma... — Ama ne güzel, ne yanık söylüyor... — Gayet muktedir bir hatip olacak. — Şüphesiz. Sokaktan gelen ses: "...Bugün varız, yarın yok! Gündüzün sonu gece. Aydınlığın sonu karanlık. Ateşin sonu kül. Hayatın sonu ölüm... Ölümden kim şüphe eder? Altınlara gark olsak , demirden, çelikten kaleler içine saklansak, mutlaka ölüm oku gelip bizi bulacak. Er geç bize yetişecek. Bu kadar muhakkak bir akibet karşısında gaflete düşen, nefsine uyan, yarını unutan insan mıdır? Hayır... Hayvandır. Nefsine uyanların, zevkten başka bir şey tanımayanların, hayvanlardan ne farkı var?" Neşet ayağa kalkar, der ki: — Bu bir feylesof! — Hem de derin... — Hem pek derin bir feylesof! — Bak, bak, ne diyor? Sokaktan gelen ses daha yanık bir hararet, daha azametli bir şiddetle: "...Merhamet, şefkat, elâlem, kimsenin umurunda değil. Sadakanın ismi unutulmuş. Yiyiniz, içiniz, keyif ediniz... Çalınız. Oynayınız. Güzel evlerin içinde, temiz karyolalarda, rahat rahat gündüz uykularına yatınız. Ah, nerede fazilet?" Niyazi oturduğu yerden: — Hem sosyalist, be! der. — Ne demek?.. — Güpe


</div>

</div>

# Comments
Results show that underlying grammar patterns can be easily extracted (huge decrease in the loss value). After that, according to the experiments, it learns harder patterns to decrease the loss further because easier patterns are already learned (grammar patterns). Those harder patterns include the semantic information of the texts. I experimented with two different layers (i.e. I experimented 1 layer and 2 layer LSTM model.).

# Results
It is clear to see that 1-layer LSTM copied the text mostly, whereas 2-layer LSTM preserves the style of the author while do not overfitting it. It is due to more complex representation of the 2-layer LSTM, which makes it harder to overfit.